In [10]:
"""
Dealing with probabilities of zero:
Laplace’s Rule of Succession adds 1 to each element of Count(Motifs). 
probabilities are now calculated with 2*n elements.
CountWithPseudocounts(Motifs) that takes a list of strings Motifs as input 
and returns the count matrix of Motifs with pseudocounts as a dictionary of lists.
"""

def count_m(motif):
    count = {} 
    t = len(motif)
    k = len(motif[0])
    for symbol in "ACGT":
        count[symbol] = [1] * k
    for i in range(t):
        for j in range(k):
            symbol = motif[i][j]
            count[symbol][j] += 1
    return count

def prob_m(motif):
    t = len(motif)
    k = len(motif[0])
    profile = count_m(motif)
    for base, l in profile.items():
        nl = [c / (t+4) for c in l]
        profile[base] = nl
    return profile

motif = ["TCGGGGGTTTTT", 
        "CCGGTGACTTAC", 
        "ACGGGGATTTTC", 
        "TTGGGGACTTTT", 
        "AAGGGGACTTCC", 
        "TTGGGGACTTCC", 
        "TCGGGGATTCAT", 
        "TCGGGGATTCCT", 
        "TAGGGGAACTAC", 
        "TCGGGTATAACC"]

motif2 = ["AACGTA",
          "CCCGTT",
          "CACCTT",
          "GGATTA",
          "TTCCGG"]

count_bm = count_m(motif)
print("Base pseudocount 1: ", count_bm)
print()
prob_bm = prob_m(motif)
print("Base pseudoprob 1: ", prob_bm)
print()
count_bm2 = count_m(motif2)
print("Base pseudocount 2: ", count_bm2)
print()
prob_bm2 = prob_m(motif2)
print("Base pseudoprob 2: ", prob_bm2)
print()


Base pseudocount 1:  {'A': [3, 3, 1, 1, 1, 1, 10, 2, 2, 2, 4, 1], 'C': [2, 7, 1, 1, 1, 1, 1, 5, 2, 3, 5, 7], 'G': [1, 1, 11, 11, 10, 10, 2, 1, 1, 1, 1, 1], 'T': [8, 3, 1, 1, 2, 2, 1, 6, 9, 8, 4, 5]}

Base pseudoprob 1:  {'A': [0.21428571428571427, 0.21428571428571427, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.7142857142857143, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.2857142857142857, 0.07142857142857142], 'C': [0.14285714285714285, 0.5, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.35714285714285715, 0.14285714285714285, 0.21428571428571427, 0.35714285714285715, 0.5], 'G': [0.07142857142857142, 0.07142857142857142, 0.7857142857142857, 0.7857142857142857, 0.7142857142857143, 0.7142857142857143, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142], 'T': [0.5714285714285714, 0.214285714285

In [14]:
"""
Function GreedyMotifSearchWithPseudocounts(Dna, k, t) that takes a list of strings Dna 
followed by integers k and t and returns the result where each profile matrix 
is generated with pseudocounts.
"""

def greedy_motif_search_pseudocounts(dna, k, t):
    best_motifs = []
    for i in range(0, t):
        best_motifs.append(dna[i][0:k])
    n = len(dna[0])
    for i in range(n-k+1):
        motifs = []
        motifs.append(dna[0][i:i+k])
        for j in range(1, t):
            profile = profile_pseudocounts(motifs[0:j])
            motifs.append(profile_kmer(dna[j], k, profile))
        if score(motifs) < score(best_motifs):
            best_motifs = motifs
    return best_motifs


# Subroutines 

def score(motifs):
    base_consensus = consensus(motifs)
    count = count_pseudocounts(motifs)
    score = 0
    i = 0
    for b in base_consensus:
        for k, l in count.items():
            if b != k:
                score += l[i] 
        i += 1
    return score

def count_pseudocounts(motifs):
    count = {} 
    t = len(motifs)
    k = len(motifs[0])
    for symbol in "ACGT":
        count[symbol] = [1] * k
    for i in range(t):
        for j in range(k):
            symbol = motifs[i][j]
            count[symbol][j] += 1
    return count

def profile_pseudocounts(motif):
    t = len(motif)
    k = len(motif[0])
    profile = count_pseudocounts(motif)
    for base, l in profile.items():
        nl = [c / (t+4) for c in l]
        profile[base] = nl
    return profile

def consensus(motifs):
    k = len(motifs[0])
    count = count_pseudocounts(motifs)
    result = ""
    for b in range(k):
        mx = 0
        frequent_symbol = ""
        for symbol in "ACGT":
            if count[symbol][b] > mx:
                mx = count[symbol][b]
                frequent_symbol = symbol
        result += frequent_symbol
    return result


def profile_kmer(text, k, profile):
    n = len(text)
    mx_p = -1
    result = ""
    for i in range(n-k+1):
        pattern = text[i:i+k]
        if prb(pattern, profile) > mx_p:
            mx_p = prb(pattern, profile)
            result = pattern
    return result

def prb(pattern, profile):
    pr = 1.
    i = 0
    for b in pattern:
        for k, l in profile.items():
            if b == k:
                pr = pr*l[i]
        i += 1
    return pr

"""*********************************************************"""

k1 = 3
t1 = 5
motif1 = ["GGCGTTCAGGCA", 
          "AAGAATCAGTCA", 
          "CAAGGAGTTCGC", 
          "CACGTCAATCAC", 
          "CAATAATATTCG"]

k2 = 5
t2 = 8
motif2 = ["AGGCGGCACATCATTATCGATAACGATTCGCCGCATTGCC",
          "ATCCGTCATCGAATAACTGACACCTGCTCTGGCACCGCTC",
          "AAGCGTCGGCGGTATAGCCAGATAGTGCCAATAATTTCCT",
          "AGTCGGTGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTG", 
          "AACCGGACGGCAACTACGGTTACAACGCAGCAAGAATATT", 
          "AGGCGTCTGTTGTTGCTAACACCGTTAAGCGACGGCAACT", 
          "AAGCGGCCAACGTAGGCGCGGCTTGGCATCTCGGTGTGTG", 
          "AATTGAAAGGCGCATCTTACTCTTTTCGCTTTCAAAAAAA"]

k3 = 5
t3 = 8
motif3 = ["GCACATCATTAAACGATTCGCCGCATTGCCTCGATAGGCG", 
          "TCATAACTGACACCTGCTCTGGCACCGCTCATCCGTCGAA", 
          "AAGCGGGTATAGCCAGATAGTGCCAATAATTTCCTTCGGC", 
          "AGTCGGTGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTG", 
          "AACCGGACGGCAACTACGGTTACAACGCAGCAAGAATATT",
          "AGGCGTCTGTTGTTGCTAACACCGTTAAGCGACGGCAACT", 
          "AAGCTTCCAACATCGTCTTGGCATCTCGGTGTGTGAGGCG", 
          "AATTGAACATCTTACTCTTTTCGCTTTCAAAAAAAAGGCG"]

k4 = 5
t4 = 8
motif4 = ["GCACATCATTATCGATAACGATTCATTGCCAGGCGGCCGC", 
          "TCATCGAATAACTGACACCTGCTCTGGCTCATCCGACCGC",
          "TCGGCGGTATAGCCAGATAGTGCCAATAATTTCCTAAGCG", 
          "GTGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTGAGTCG",
          "GACGGCAACTACGGTTACAACGCAGCAAGAATATTAACCG", 
          "TCTGTTGTTGCTAACACCGTTAAGCGACGGCAACTAGGCG", 
          "GCCAACGTAGGCGCGGCTTGGCATCTCGGTGTGTGAAGCG", 
          "AAAGGCGCATCTTACTCTTTTCGCTTTCAAAAAAAAATTG"]

k4 = 3
t4 = 8
motif4 = ["GCACATCATTATCGATAACGATTCATTGCCAGGCGGCCGC", 
          "TCATCGAATAACTGACACCTGCTCTGGCTCATCCGACCGC",
          "TCGGCGGTATAGCCAGATAGTGCCAATAATTTCCTAAGCG",
          "GTGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTGAGTCG", 
          "GACGGCAACTACGGTTACAACGCAGCAAGAATATTAACCG",
          "TCTGTTGTTGCTAACACCGTTAAGCGACGGCAACTAGGCG", 
          "GCCAACGTAGGCGCGGCTTGGCATCTCGGTGTGTGAAGCG", 
          "AAAGGCGCATCTTACTCTTTTCGCTTTCAAAAAAAAATTG"]


print("Greedy motif search with pseudocounts: ", greedy_motif_search_pseudocounts(motif1, k1, t1))
print()
print("Greedy motif search with pseudocounts: ", greedy_motif_search_pseudocounts(motif2, k2, t2))
print()
print("Greedy motif search with pseudocounts: ", greedy_motif_search_pseudocounts(motif3, k3, t3))
print()
print("Greedy motif search with pseudocounts: ", greedy_motif_search_pseudocounts(motif4, k4, t4))
print()


Greedy motif search with pseudocounts:  ['TTC', 'ATC', 'TTC', 'ATC', 'TTC']

Greedy motif search with pseudocounts:  ['AGGCG', 'ATCCG', 'AAGCG', 'AGTCG', 'AACCG', 'AGGCG', 'AGGCG', 'AGGCG']

Greedy motif search with pseudocounts:  ['AGGCG', 'TGGCA', 'AAGCG', 'AGGCA', 'CGGCA', 'AGGCG', 'AGGCG', 'AGGCG']

Greedy motif search with pseudocounts:  ['GGC', 'GGC', 'GGC', 'GGC', 'GGC', 'GGC', 'GGC', 'GGC']



In [16]:
"""
Find motifs in the DosR dataset with k-mer length equal to 15
"""

DosR_dna = ["GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC",
            "CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG",
            "ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC",
            "GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC",
            "GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG",
            "CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA",
            "GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA",
            "GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG",
            "GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG",
            "TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"]
k = 15
t = len(DosR_dna) 

Dos_R_motifs = greedy_motif_search_pseudocounts(DosR_dna, k, t)
print()
print("Greedy motif search with pseudocounts for Dos_R: ", Dos_R_motifs)
print()
print("Score: ", score(Dos_R_motifs))


Greedy motif search with pseudocounts for Dos_R:  ['GGACTTCAGGCCCTA', 'GGTCAAACGACCCTA', 'GGACGTAAGTCCCTA', 'GGATTACCGACCGCA', 'GGCCGAACGACCCTA', 'GGACCTTCGGCCCCA', 'GGACTTCTGTCCCTA', 'GGACTTTCGGCCCTG', 'GGACTAACGGCCCTC', 'GGACCGAAGTCCCCG']

Score:  80
